# A coupled PO₄–POP-DOP-dFe model

 We consider a simple model for the cycling of phosphorus with 2 state variables consisting of phosphate (PO₄) AKA dissolved inorganic phosphorus (DIP), particulate organic phosphorus (POP), and Dissolved Organic phosphorus (DOP).
 The dissolved phases are transported by advection and diffusion whereas the particulate phase sinks rapidly down the water column without any appreciable transport by the circulation.

 The governing equations that couple the 3D concentration fields of DIP, DOP, and POP, denoted $x_\mathsf{DIP}$, $x_\mathsf{DOP}$, and $x_\mathsf{POP}$, respectively, are:

$$\left[\frac{\partial}{\partial t} + \nabla \cdot (\boldsymbol{u} + \mathbf{K}\nabla )\right] x_\mathsf{DIP} = -U(\mathsf{min}(x_\mathsf{DIP},x_\mathsf{DFE})) + R_{\mathsf{POP}}(x_\mathsf{POP}) + R_{\mathsf{DOP}}(x_\mathsf{DOP}) ,$$

$$\left[\frac{\partial}{\partial t} + \nabla \cdot (\boldsymbol{u} + \mathbf{K}\nabla )\right] x_\mathsf{DOP} = \lambda_{DOP}U(\mathsf{min}(x_\mathsf{DIP},x_\mathsf{DFE})) - R_{\mathsf{DOP}}(x_\mathsf{DOP}),$$

$$\left[\frac{\partial}{\partial t} + \nabla \cdot \boldsymbol{w}\right] x_\mathsf{POP} = (1-\lambda_{DOP})U(\mathsf{min}(x_\mathsf{DIP},x_\mathsf{DFE})) - R_{\mathsf{POP}}(x_\mathsf{POP}),$$

$$\left[\frac{\partial}{\partial t} + \nabla \cdot (\boldsymbol{u} + \mathbf{K}\nabla )\right] x_\mathsf{DFE} = S_{\mathsf{Fe}} -R_{\mathsf{Fe:P}}U(\mathsf{min}(x_\mathsf{DIP},x_\mathsf{DFE})) + R_{\mathsf{Fe:P}}R_{\mathsf{POP}}(x_\mathsf{POP}) + R_{\mathsf{Fe:P}}R_{\mathsf{DOP}}(x_\mathsf{DOP})- S_\mathsf{Scav},$$

 The $\nabla \cdot \left[ \boldsymbol{u} - \mathbf{K} \nabla \right]$ and $\nabla \cdot \boldsymbol{w}$ operators represent the ocean circulation and the sinking of particles, respectively.
 ([Tracer transport operators are described in the documentation](@ref tracer-transport-operators).)

 The function $U$ represents the biological uptake of DIP by phytoplankton, which we model here as

 $$U(x_\mathsf{DIP}) = \frac{x_\mathsf{DIP}}{\tau_\mathsf{DIP}} \, \mathsf{min}\left(\frac{x_\mathsf{DIP}}{x_\mathsf{DIP} + k_{\mathsf{DIP}}},\frac{x_\mathsf{DFE}}{x_\mathsf{DFE} + k_{\mathsf{Fe}}}\right) \, (z < z_0),$$

 with the timescale, $\tau$, the half-saturation rate $k$, and the depth $z_0$ as parameters. A fraction, $\lambda_{DOP}$, of production is exuded as DOP, while the remainder forms sinking POP. 

The function $R_{POP}$ defines the remineralization rate of POP, which converts POP back into DIP.
 For the remineralization, we simply use a linear rate constant, i.e.,

 $$R(x_\mathsf{POP}) = \frac{x_\mathsf{POP}}{\tau_\mathsf{POP}}.$$

The function $R_{DOP}$ defines the remineralization rate of DOP, which converts DOP back into DIP.
 For the remineralization, we simply use a linear rate constant, i.e.,

 $$R(x_\mathsf{DOP}) = \frac{x_\mathsf{DOP}}{\tau_\mathsf{DOP}}.$$

 We start by telling Julia we want to use the AIBECS and the OCIM2 circulation.

In [1]:
using AIBECS
grd, T_OCIM = OCIM2.load()
T_DIP(p) = T_OCIM
T_DOP(p) = T_OCIM
T_DFE(p) = T_OCIM

┌ Info: You are about to use the OCIM2_CTL_He model.
│ If you use it for research, please cite:
│ 
│ - DeVries, T., & Holzer, M. (2019). Radiocarbon and helium isotope constraints on deep ocean ventilation and mantle‐³He sources. Journal of Geophysical Research: Oceans, 124, 3036–3057. https://doi.org/10.1029/2018JC014716
│ 
│ You can find the corresponding BibTeX entries in the CITATION.bib file
│ at the root of the AIBECS.jl package repository.
│ (Look for the "DeVries_Holzer_2019" key.)
└ @ AIBECS.OCIM2 /Users/jml1/Dropbox (MIT)/Applications/Julia/AIBECS.jl/src/OCIM2.jl:113


T_DFE (generic function with 1 method)

For the sinking of particles, we use the `transportoperator` function

In [2]:
T_POP(p) = transportoperator(grd, z -> w(z,p))

T_POP (generic function with 1 method)

 for which we need to define the sinking speed `w(z,p)` as a function of depth `z` and of the parameters `p`.
 Following the assumption that $w(z) = w_0 + w' z$ increases linearly with depth, we write it as

In [3]:
function w(z,p)
    @unpack w₀, w′ = p
    return @. w₀ + w′ * z
end

w (generic function with 1 method)

 #### Sources of iron

In [4]:
s_A_2D = AeolianSources.load()
s_dust_2D_monthly = s_A_2D[:Dust] # kg m⁻² s⁻¹
s_dust_2D_annual = permutedims(dropdims(sum(s_dust_2D_monthly, dims=3), dims=3), (2,1)) / 12
s_dust_2D = regrid(s_dust_2D_annual, s_A_2D[:lat], s_A_2D[:lon], grd)
s_dust_3D = zeros(size(grd)...)
s_dust_3D[:,:,1] .= s_dust_2D
s_dust_3D = ustrip.(upreferred.(s_dust_3D * u"kg/m^2/s" ./ grd.δz_3D))
s_dust = vectorize(s_dust_3D, grd) # which is the same as `s_dust_3D[iswet(grd)]`

# Convert dust to mol iron input
s_dust = ustrip.((s_dust * u"kg/m^3/s" .|> u"g/m^3/s") .* (0.035 * Unitful.NoUnits) .* (1.0/58.0) * u"mol/g")
s_hvt  = s_dust.*0.0
s_sed  = s_dust.*0.0

Sfe = s_dust + s_hvt + s_sed
#function Sfe(x, p)
#    return s_dust + s_hvt + s_sed
#end

┌ Info: You are about to use the Chien et al. (2016) data for aeolian deposition.
│ If you use it for research, please cite:
│ 
│ - Chien, C.-T., K. R. M. Mackey, S. Dutkiewicz, N. M. Mahowald, J. M. Prospero, and A. Paytan (2016), Effects of African dust deposition on phytoplankton in the western tropical Atlantic Ocean off Barbados, Global Biogeochem. Cycles, 30, doi:10.1002/2015GB005334.
│ 
│ You can find the corresponding BibTeX entries in the CITATION.bib file
│ at the root of the AIBECS.jl package repository.
│ (Look for the "Chien_etal_2016" key.)
└ @ AIBECS.AeolianSources /Users/jml1/Dropbox (MIT)/Applications/Julia/AIBECS.jl/src/aeolian_sources.jl:66


200160-element Array{Float64,1}:
 2.225648221366747e-15
 3.69999608559021e-15
 5.8307518383367515e-15
 7.64964551017875e-15
 1.0446635652625328e-14
 1.3590464191396246e-14
 1.9342010559332763e-14
 2.837171519543432e-14
 2.7134308091540578e-14
 2.8058743656156848e-14
 2.962856784609998e-14
 2.8419833494212843e-14
 2.5033021970871725e-14
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

#### Sink of iron due to scavenging

In [5]:
function Scav(DFE, p)
    @unpack τfescav, τdfe, D̅F̅E̅ = p
    return @. (DFE/τfescav) + (D̅F̅E̅≤DFE) * ((D̅F̅E̅ - DFE) / τdfe)
end

Scav (generic function with 1 method)

 #### Uptake of DIP
 For the uptake, $U$, we write

In [11]:
z = depthvec(grd)
function U(DIP, DOP, POP, DFE, p)
    @unpack τBP, kDIP, kFE, z₀ = p
    thx = DIP/(DIP+kDIP)
    thy = DFE/(DFE+kFE )
    thaux = tanh( (thx-thy)*1.e6 )
    nutlimit= ( 1.0 - thaux ) * thx * 0.5 + ( 1.0 + thaux ) * thy * 0.5
    return @. 1.0/τBP * nutlimit * (z≤z₀) * (DIP≥0) * (DFE≥0)
end

U (generic function with 1 method)

where we have "unpacked" the parameters to make the code clearer and as close to the mathematical equation as possible.
(Note we have also added a constraint that `x` must be positive for uptake to happen.)

##### Remineralization 

In [12]:
function RPOP(x,p)
    @unpack τPOP = p
    return x / τPOP
end

function RDOP(x,p)
    @unpack τDOP = p
    return x / τDOP
end

RDOP (generic function with 1 method)

##### Net sources and sinks

We lump the sources and sinks into `G` functions for DIP, DOP, and POP.

In [13]:
function G_DIP(DIP, DOP, POP, DFE, p)
    @unpack D̅I̅P̅, τpo4 = p
    return @. -$U(DIP, DOP, POP, DFE, p) + $RDOP(DOP,p) + $RPOP(POP,p) + (D̅I̅P̅ - DIP) / τpo4
end

function G_DOP(DIP, DOP, POP, DFE, p)
    @unpack λDOP = p
    return U(DIP, DOP, POP, DFE, p) * λDOP - RDOP(DOP,p)
end

function G_POP(DIP, DOP, POP, DFE, p)
    @unpack λDOP = p
    return U(DIP, DOP, POP, DFE, p) * (1.0-λDOP) - RPOP(POP,p)
end

function G_DFE(DIP, DOP, POP, DFE, p)
    @unpack Rfep = p
    return Sfe(DFE, p) + RDOP(DOP,p)*Rfep + RPOP(POP,p)*Rfep - U(DIP, DOP, POP, DFE, p)*Rfep - Scav(DFE, p)
end

G_DFE (generic function with 1 method)

where we have imposed a slow restoring of DIP to the global mean `D̅I̅P̅` to prescribe the global mean concentration.
(The `$` signs in front of `U` and `R` protect them from the broadcast macro `@.`)

We now define and build the parameters.

In this tutorial we will specify some initial values for the parameters
and also include units.

In [23]:
import AIBECS: @units, units
import AIBECS: @initial_value, initial_value
@initial_value @units struct PmodelParameters{U} <: AbstractParameters{U}
    w₀::U      |  0.64 | u"m/d"
    w′::U      |  0.13 | u"m/d/m"
    τBP::U     | 230.0 | u"d"
    kDIP::U    |  6.62 | u"μmol/m^3"
    kFE::U     |  0.1  | u"nmol/m^3"
    z₀::U      |  80.0 | u"m"
    τPOP::U    |   5.0 | u"d"
    τDOP::U    | 180.0 | u"d"    
    τpo4::U    |   1.0 | u"Myr"
    D̅I̅P̅::U     |  2.12 | u"mmol/m^3"
    λDOP::U    |  0.67 | Unitful.NoUnits
    Rfep::U    |  1e-3 | u"mol/mol"
    D̅F̅E̅::U     |  1.2  | u"μmol/m^3"
    τdfe::U    |  1.0  | u"d"
    τfescav::U | 120.0 | u"d"
end

initial_value (generic function with 24 methods)

Finally, thanks to the initial values we provided, we can instantiate the parameter vector succintly as

In [24]:
@time p = PmodelParameters()

  0.007923 seconds (11.73 k allocations: 715.167 KiB)



│ Row │ Symbol  │ Value   │ Initial value │ Unit     │
│     │ Symbol  │ Float64 │ Float64       │ Unitful… │
├─────┼─────────┼─────────┼───────────────┼──────────┤
│ 1   │ w₀      │ 0.64    │ 0.64          │ m d⁻¹    │
│ 2   │ w′      │ 0.13    │ 0.13          │ d⁻¹      │
│ 3   │ τBP     │ 230.0   │ 230.0         │ d        │
│ 4   │ kDIP    │ 6.62    │ 6.62          │ μmol m⁻³ │
│ 5   │ kFE     │ 0.1     │ 0.1           │ nmol m⁻³ │
│ 6   │ z₀      │ 80.0    │ 80.0          │ m        │
│ 7   │ τPOP    │ 5.0     │ 5.0           │ d        │
│ 8   │ τDOP    │ 180.0   │ 180.0         │ d        │
│ 9   │ τpo4    │ 1.0     │ 1.0           │ Myr      │
│ 10  │ D̅I̅P̅     │ 2.12    │ 2.12          │ mmol m⁻³ │
│ 11  │ λDOP    │ 0.67    │ 0.67          │          │
│ 12  │ Rfep    │ 0.001   │ 0.001         │          │
│ 13  │ D̅F̅E̅     │ 1.2     │ 1.2           │ μmol m⁻³ │
│ 14  │ τdfe    │ 1.0     │ 1.0           │ d        │
│ 15  │ τfescav │ 120.0   │ 120.0         │ d        │

PmodelParameters{Float64}

We generate the state function `F` and its Jacobian `∇ₓF`,

In [25]:
nb = sum(iswet(grd))
F, ∇ₓF = state_function_and_Jacobian((T_DIP, T_DOP, T_POP, T_DFE), (G_DIP, G_DOP, G_POP, G_DFE), nb)

(AIBECS.var"#F#31"{Tuple{typeof(T_DIP),typeof(T_DOP),typeof(T_POP),typeof(T_DFE)},AIBECS.var"#tracer#26"{Int64,Int64},AIBECS.var"#G#29"{Tuple{typeof(G_DIP),typeof(G_DOP),typeof(G_POP),typeof(G_DFE)},AIBECS.var"#tracers#25"{Int64,Int64}}}((T_DIP, T_DOP, T_POP, T_DFE), AIBECS.var"#tracer#26"{Int64,Int64}(200160, 4), AIBECS.var"#G#29"{Tuple{typeof(G_DIP),typeof(G_DOP),typeof(G_POP),typeof(G_DFE)},AIBECS.var"#tracers#25"{Int64,Int64}}((G_DIP, G_DOP, G_POP, G_DFE), AIBECS.var"#tracers#25"{Int64,Int64}(200160, 4))), AIBECS.var"#∇ₓF#34"{AIBECS.var"#T#27"{Tuple{typeof(T_DIP),typeof(T_DOP),typeof(T_POP),typeof(T_DFE)}},AIBECS.var"#∇ₓG#33"{Tuple{typeof(G_DIP),typeof(G_DOP),typeof(G_POP),typeof(G_DFE)},Int64,Int64}}(AIBECS.var"#T#27"{Tuple{typeof(T_DIP),typeof(T_DOP),typeof(T_POP),typeof(T_DFE)}}((T_DIP, T_DOP, T_POP, T_DFE)), AIBECS.var"#∇ₓG#33"{Tuple{typeof(G_DIP),typeof(G_DOP),typeof(G_POP),typeof(G_DFE)},Int64,Int64}((G_DIP, G_DOP, G_POP, G_DFE), 200160, 4)))

generate the steady-state problem,

In [26]:
@unpack D̅I̅P̅, D̅F̅E̅ = p
x = [D̅I̅P̅ * ones(nb); zeros(2nb); D̅F̅E̅ * ones(nb)] # initial guess
prob = SteadyStateProblem(F, ∇ₓF, x, p)

SteadyStateProblem with uType Array{Float64,1}
u0: [0.0021200000000000004, 0.0021200000000000004, 0.0021200000000000004, 0.0021200000000000004, 0.0021200000000000004, 0.0021200000000000004, 0.0021200000000000004, 0.0021200000000000004, 0.0021200000000000004, 0.0021200000000000004  …  1.2e-6, 1.2e-6, 1.2e-6, 1.2e-6, 1.2e-6, 1.2e-6, 1.2e-6, 1.2e-6, 1.2e-6, 1.2e-6]

and solve it

In [ ]:
@time s = solve(prob, CTKAlg()).u

We can look at different the DIP, DOP, and POP fields using the Plots.jl recipes.

In [ ]:
DIP, DOP, POP, DFE = state_to_tracers(s, grd) # unpack tracers

We can plot the concentration of DIP, DOP, and DFE at a given depth via, e.g.,

In [ ]:
using Plots
plothorizontalslice(DIP * u"mol/m^3" .|> u"μM", grd, depth=500u"m", color=:viridis)

In [ ]:
plothorizontalslice(DOP * u"mol/m^3" .|> u"μM", grd, depth=500u"m", color=:viridis)

In [ ]:
plothorizontalslice(DFE * u"mol/m^3" .|> u"nM", grd, depth=500u"m", color=:viridis)

Or have a look at a map of the uptake at the surface

In [ ]:
plotverticalintegral(U(DIP,p) * u"mol/m^3/s" .|> u"mmol/yr/m^3", grd, color=:algae)

Or look at what is exported below 500 m

In [ ]:
plothorizontalslice(POP .* w(z,p) * u"mol/m^3*m/s" .|> u"mmol/yr/m^2", grd, depth=500u"m", color=:inferno, rev=true)